In [1]:
import re
import requests
import pdfplumber
import pandas as pd
from collections import namedtuple

In [2]:
Rep = namedtuple('Rep', 'source Amt ')

In [3]:
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [4]:
ap = 'Aggregate under online shopping.pdf'

In [5]:
with pdfplumber.open(ap) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()

In [6]:
print(text)

Aggregate under online shopping
· ALERT:You've spent Rs.2.00 via Debit Card xx2217 at AMAZON PAY INDIA PRIVA on
2021-03-25:02:10:37.Avl Bal Rs.1091.07.Not you?Call 18002586161.
Aggregate under bank withdrawal
· ALERT:You've withdrawn Rs.500.00 via Debit Card xx1135 at +UBI NOIDA MAIN on
2021-06-06:20:27:20.Avl Bal Rs.854.14.Not you?Call 18002586161.
Aggregate under UPI
· HDFC Bank: Rs 1.00 debited from a/c **1135 on 06-08-21 to VPA
kshreyansnahar@okhdfcbank (UPI Ref No 121554846690). Not you? Call on
18002586161 to report
Aggregate under food
· Rs.327.25 on Zomato charged via Simpl.
Food, groceries, commute, or medicines. Buy Now, Pay Later via Simpl. Know More:
https://app.getsimpl.com/Jrf0zPQaFt- Simpl Pay
· Payment of Rs. 370.6 using OlaMoney Postpaid for your transaction fzip-ndco-5wbi on
Zomato is successful. Not you? Write to us at support@olamoney.com
Aggregate under credit applications
· HDFC Bank: Rs 393.66 debited from a/c **1135 on 20-07-21 to VPA simpl@axb(UPI Ref
No 140653

In [7]:
new = re.compile(r'^[Agg].*')

In [8]:
for line in text.split('\n'):
    if new.match(line):
        print(line)

Aggregate under online shopping
Aggregate under bank withdrawal
Aggregate under UPI
Aggregate under food
Aggregate under credit applications


In [9]:
for line in text.split('\n'):
    if new.match(line):
        char1,char2, *source = line.split()
        source = ' '.join(source)
print(source)

credit applications


In [14]:
inv_line_re = re.compile(r'([\d,]+\.\d{2}) ')

In [18]:
line_items = []
for line in text.split('\n'):
    if new.match(line):
        char1,char2, *source = line.split()
        source = ' '.join(source)    
    
    line = inv_line_re.search(line)
    if line:
        Amt = line.group(1)
        line_items.append(Rep(source,Amt))

In [19]:
df = pd.DataFrame(line_items)

In [20]:
df.head()

,source,Amt
0,online shopping,2.00
1,bank withdrawal,500.00
2,UPI,1.00
3,food,327.25
4,credit applications,393.66


In [23]:
df['Amt'] = df['Amt'].map(lambda x: float(x.replace(',', '')))

In [24]:
df.sum()

source    online shoppingbank withdrawalUPIfoodcredit ap...
Amt                                                 1369.51
dtype: object